<h1>Projket staż M-bank 2021</h1>
<h2>Mateusz Jakubczak</h2>


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import phik
import statsmodels.api as sm
from scipy import stats
import math
from pandas_profiling import ProfileReport
import statsmodels.stats.weightstats as st 

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython.display import display, HTML

### Pierwsze co należy zrobić w analizie to przygotowac dane.


In [2]:
df =pd.read_csv("BankChurners.csv")
df.head()

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,...,12691.0,777,11914.0,1.335,1144,42,1.625,0.061,0.000093,0.99991
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,...,8256.0,864,7392.0,1.541,1291,33,3.714,0.105,0.000057,0.99994
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,...,3418.0,0,3418.0,2.594,1887,20,2.333,0.000,0.000021,0.99998
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,...,3313.0,2517,796.0,1.405,1171,20,2.333,0.760,0.000134,0.99987
4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,...,4716.0,0,4716.0,2.175,816,28,2.500,0.000,0.000022,0.99998


## Mamy dwie zmienne gdzie, patrząc na ich nazwę są wynikiem modelu.
Ponieważ nie znamy samych parametrów tego modelu poza tym jakie zmienne zostały w nim urzyte nie możemy użyć tych zmiennych, jak i nie ma ich w specyfikacji samego zadania, dlatego pozbywamy się tych zmiennych. Wyrzucamy równiez zmienną CLIENTNUM, która stanowi tylko index dla naszych danych. 

In [3]:
df = df.drop(["Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2",          "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1","CLIENTNUM"],
             axis = 1)

Mamy parę nie zgodnych typów kategorycznych rozpoznacych jako obiekty. Poprawiamy to

In [4]:
df = df.convert_dtypes()

In [5]:
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10127 entries, 0 to 10126
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Attrition_Flag            10127 non-null  string 
 1   Customer_Age              10127 non-null  Int64  
 2   Gender                    10127 non-null  string 
 3   Dependent_count           10127 non-null  Int64  
 4   Education_Level           10127 non-null  string 
 5   Marital_Status            10127 non-null  string 
 6   Income_Category           10127 non-null  string 
 7   Card_Category             10127 non-null  string 
 8   Months_on_book            10127 non-null  Int64  
 9   Total_Relationship_Count  10127 non-null  Int64  
 10  Months_Inactive_12_mon    10127 non-null  Int64  
 11  Contacts_Count_12_mon     10127 non-null  Int64  
 12  Credit_Limit              10127 non-null  float64
 13  Total_Revolving_Bal       10127 non-null  Int64  
 14  Avg_Op

None

Sprawdzamy czy są duplikaty oraz puste warotścoi

In [6]:

display(pd.DataFrame([df.duplicated().sum()],columns=["Ilość duplikatów"]).style.hide_index())

Ilość duplikatów
0


Dane sa czyse i możemy z nimi pracować

# Wizualizacja danych

In [7]:
#profile = ProfileReport(df, config_file="config.yml")


In [8]:
#profile.to_notebook_iframe()


## Wnioski z eksprolacji

1. Mamy podejrzanie dużą ilość rachunków które mają daokładnie 3 lata (36 miesięcy)
2. Wartości typu NaN są nie prowidlo zakfalifikowane jako "Unknow"
3. Nasza zmienna endogeniczna jest nie zbalansowana
4. Mamy podejrzanie wysoką ilość klientów z wiekiem około 48 lat
5. Zmienna Card_Category jest wysoce nie zbalansaowan, 20 platynowych kart do 9.5 tyś niebieskich
6. Credit limit posiada dystrybucje z grubym ogonem
7. Większość zmiennych nie przyjmuje rozkładu normalnego

## Sprawdzmy korelacje naszej  zmiennej endogenicznej
poniża kolumna przedstawia korelecej typu phik, im warość jest bliższa 1 tym bardziej skorelowane ze sobą są dwie wartości. 

In [9]:

for c in ["Attrition_Flag","Gender","Education_Level","Marital_Status","Income_Category","Card_Category"]:
    df[c] = df[c].replace("Unknown",np.nan,method="pad")
    df[c] = df[c].astype("category")

In [10]:
df["Education_Level"].value_counts()

Graduate         3128
High School      2013
Uneducated       1487
College          1013
Post-Graduate     516
Doctorate         451
Name: Education_Level, dtype: int64

In [11]:
display(df.isna().sum().to_frame("Ilośc NA"))

,Ilośc NA
Attrition_Flag,0
Customer_Age,0
Gender,0
Dependent_count,0
Education_Level,1519
Marital_Status,749
Income_Category,1112
Card_Category,0
Months_on_book,0
Total_Relationship_Count,0


### Mamy parę wartości typu NA w zmiennych kategorycznych. W dalszej części analizy zdecydujemy co z nimi zrobimy, bo może się okazać że nie wykorzystamy tych kolumn do dalszej analizy. 

# Obecny stan kolumn

In [12]:
corr = df.phik_matrix()
display(corr["Attrition_Flag"].sort_values(ascending = False).to_frame())

interval columns not set, guessing: ['Customer_Age', 'Dependent_count', 'Months_on_book', 'Total_Relationship_Count', 'Months_Inactive_12_mon', 'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal', 'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt', 'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio']


,Attrition_Flag
Attrition_Flag,1.000000
Total_Trans_Ct,0.592470
Total_Revolving_Bal,0.522422
Total_Trans_Amt,0.423372
Total_Ct_Chng_Q4_Q1,0.409053
Avg_Utilization_Ratio,0.314541
Total_Relationship_Count,0.230833
Contacts_Count_12_mon,0.223812
Total_Amt_Chng_Q4_Q1,0.184180
Months_Inactive_12_mon,0.183537


### Wnioski z korelacji 
1. Mamy wysoką korelacje z ilością wykonanych tranzakcji a zmienną endogeniczną 
2. Zmienne z NA mają bardzo niską korelacje z zmienną endogeniczną

# Badnaie hipotez 
1. Posiadacze wyszej kalsy karty mają mniejszą szansę na zamknięcie konta
2. Posiadacze konta wykonujący dużo tranzakcj mają mniejszą szanse na zamknięcie konta 
3. Posiadacze konta przez długi czas maja mniejszą szanse odejścia 

In [13]:
#Hipoteza 1 
df.Card_Category.value_counts()
df_h1 = df.Card_Category == "Blue"
df_h1_prem = df.Card_Category == "Platinum"

Blue        9436
Silver       555
Gold         116
Platinum      20
Name: Card_Category, dtype: int64

In [14]:
df["Attrition_Flag_onehot"] = pd.get_dummies(df["Attrition_Flag"]).iloc[:,0]

df[["Attrition_Flag","Attrition_Flag_onehot"]].head()

,Attrition_Flag,Attrition_Flag_onehot
0,Existing Customer,0
1,Existing Customer,0
2,Existing Customer,0
3,Existing Customer,0
4,Existing Customer,0


## Z one hot encodingu wynika że obecny konsument ma przypisane 0, natomiast z zamkniętym kontem ma wartość 1 

In [15]:
prob_blue = df[df_h1]["Attrition_Flag"].value_counts()
d_blue = {
"prawdobienstwo odejścia":prob_blue.iloc[1] / prob_blue.iloc[0],
 "Liczebnosc populacji":prob_blue.iloc[1] + prob_blue.iloc[0]
}
display(pd.DataFrame.from_dict(d_blue,orient='index'))

,0
prawdobienstwo odejścia,0.191866
Liczebnosc populacji,9436.000000


In [16]:
prob_rest = df[~df_h1]["Attrition_Flag"].value_counts()
d_rest = {
"prawdobienstwo odejścia":prob_rest.iloc[1] / prob_rest.iloc[0],
 "Liczebnosc populacji":prob_rest.iloc[1] + prob_rest.iloc[0]
}
display(pd.DataFrame.from_dict(d_rest,orient='index'))

,0
prawdobienstwo odejścia,0.185249
Liczebnosc populacji,691.000000


In [17]:

display(pd.DataFrame.from_dict({
    "P-value dla t testu, porównanie miedzy Blue a Platinum":
st.ttest_ind(df[df_h1_prem]["Attrition_Flag_onehot"],df[df_h1]["Attrition_Flag_onehot"], alternative="larger")[1]
}, orient='index'))

,0
"P-value dla t testu, porównanie miedzy Blue a Platinum",0.139738


In [18]:

display(pd.DataFrame.from_dict({
    "P-value dla t testu, porównanie miedzy Blue a pozostałymi":
st.ttest_ind(df[~df_h1]["Attrition_Flag_onehot"],df[df_h1]["Attrition_Flag_onehot"], alternative="larger")[1]
}, orient='index'))

,0
"P-value dla t testu, porównanie miedzy Blue a pozostałymi",0.626887


## Możemy odrzucic hipotezę że klasa wpływa na szansę zamknięcia konta 

# Hipoteza 2  
zakładamy że osoby wykonującę dużo tranzakcji znajdują się w 90 percentylu wykonanych tranzakcji(Wartość percentyla należało by wyestymować budując model ale to stanie odbędzie się w dlaszych krokach analizy). 

### Ponieważ sama ilość trazakcji nie wiele nam mówi i jest obciązona poprzez wartość tych trazakcji, do analityki stworzymi zmieną opisującą średnią wartość tranzakcji. Sprawdzmy czy taka zmienna lepiej koreluje z 

In [19]:
df["Avg_Trans_Amt"] = df.Total_Trans_Amt/df.Total_Trans_Ct


In [20]:

corr = df[["Attrition_Flag_onehot","Total_Trans_Ct","Avg_Trans_Amt","Total_Trans_Amt"]].corr(method='kendall')
display(corr["Attrition_Flag_onehot"].to_frame())

,Attrition_Flag_onehot
Attrition_Flag_onehot,1.000000
Total_Trans_Ct,-0.309061
Avg_Trans_Amt,-0.005430
Total_Trans_Amt,-0.182744


## Zmianna ta zachowuję się o wiele gorzej niż odzielnie liczba tranzakcji oraz wartość tranzakcji. Wybieramy do dalszej analizy ilość przeprowadzonych tranzakcji. 


In [21]:
q_90 = df.Total_Trans_Ct.quantile(0.9)

display(pd.DataFrame([q_90],columns=["Wartość 90 percentyla"]).style.hide_index())


Wartość 90 percentyla
92.000000


In [22]:
df_h2 =  df.Total_Trans_Ct >= q_90

In [23]:
prob_above_q90 = df[df_h2]["Attrition_Flag"].value_counts()
d_above = {
"prawdobienstwo odejścia":prob_above_q90.iloc[1] / prob_above_q90.iloc[0],
 "Liczebnosc populacji":prob_above_q90.iloc[1] + prob_above_q90.iloc[0]
}
display(pd.DataFrame.from_dict(d_above,orient='index'))

,0
prawdobienstwo odejścia,0.00095
Liczebnosc populacji,1054.00000


In [24]:
prob_bellow_q90 = df[~df_h2]["Attrition_Flag"].value_counts()
d_bellow = {
"prawdobienstwo odejścia":prob_bellow_q90.iloc[1] / prob_bellow_q90.iloc[0],
 "Liczebnosc populacji":prob_bellow_q90.iloc[1] + prob_bellow_q90.iloc[0]
}
display(pd.DataFrame.from_dict(d_bellow,orient='index'))

,0
prawdobienstwo odejścia,0.218343
Liczebnosc populacji,9073.000000


In [25]:
display(pd.DataFrame.from_dict({
    "P-value dla t testu, porównanie miedzy powyżej 90 percentyla i poniżej":
st.ttest_ind(df[~df_h2]["Attrition_Flag_onehot"],df[df_h2]["Attrition_Flag_onehot"],
             alternative="larger")[1]
}, orient='index'))

,0
"P-value dla t testu, porównanie miedzy powyżej 90 percentyla i poniżej",3.747446e-51


### P-value jest równe 0,
co daje nam pewność o odrzenie H_0 dla t testu. Prawdopodnieństo odejścia jest zdecydowanie niższe dla osób wykonujaących dużo tranzakcji. 

### Sprawdzmy jeszcze hipotezę czy wykonywanie małej ilości tranzakcji zwiększa szanse na odejście 

In [27]:
q_10 = df.Total_Trans_Ct.quantile(0.1)

display(pd.DataFrame([q_10],columns=["Wartość 10 percentyla"]).style.hide_index())


Wartość 10 percentyla
33.000000


In [30]:
df_h2_1 =  df.Total_Trans_Ct <= q_10

In [31]:
prob_bellow_q10 = df[df_h2_1]["Attrition_Flag"].value_counts()
d_above = {
"prawdobienstwo odejścia":prob_bellow_q10.iloc[1] / prob_bellow_q10.iloc[0],
 "Liczebnosc populacji":prob_bellow_q10.iloc[1] + prob_bellow_q10.iloc[0]
}
display(pd.DataFrame.from_dict(d_above,orient='index'))

,0
prawdobienstwo odejścia,0.362784
Liczebnosc populacji,1018.000000


In [32]:
prob_above_q10 = df[~df_h2_1]["Attrition_Flag"].value_counts()
d_bellow = {
"prawdobienstwo odejścia":prob_above_q10.iloc[1] / prob_above_q10.iloc[0],
 "Liczebnosc populacji":prob_above_q10.iloc[1] + prob_above_q10.iloc[0]
}
display(pd.DataFrame.from_dict(d_bellow,orient='index'))

,0
prawdobienstwo odejścia,0.1749
Liczebnosc populacji,9109.0000


In [34]:
display(pd.DataFrame.from_dict({
    "P-value dla t testu, porównanie miedzy powyżej 90 percentyla i poniżej":
st.ttest_ind(df[~df_h2_1]["Attrition_Flag_onehot"],df[df_h2_1]["Attrition_Flag_onehot"],
             alternative="smaller")[1]
}, orient='index'))

,0
"P-value dla t testu, porównanie miedzy powyżej 90 percentyla i poniżej",1.648878e-22


### P-value znów równe 0, co oznacza że odrzucamy  H_0 dla t testu i możemy powiedziec że : 

## Im więcej tranzakcji wykonuje dana osoba tym mniejsza jest jej szansa na odejście. 

# Hipoteza 3 
### Posiadacze konta przez długi czas maja mniejszą szanse odejścia 
Robi takie same założenia jak testując wcześniejsze hipotezy 

In [36]:
df.Months_on_book.describe()

count    10127.000000
mean        35.928409
std          7.986416
min         13.000000
25%         31.000000
50%         36.000000
75%         40.000000
max         56.000000
Name: Months_on_book, dtype: float64

In [38]:
q_90 = df.Months_on_book.quantile(0.9)

display(pd.DataFrame([q_90],columns=["Wartość 90 percentyla"]).style.hide_index())

df_h3 = df.Months_on_book >= q_90

Wartość 90 percentyla
46.000000


In [39]:
prob_above_q90 = df[df_h3]["Attrition_Flag"].value_counts()
d_above = {
"prawdobienstwo odejścia":prob_above_q90.iloc[1] / prob_above_q90.iloc[0],
 "Liczebnosc populacji":prob_above_q90.iloc[1] + prob_above_q90.iloc[0]
}
display(pd.DataFrame.from_dict(d_above,orient='index'))

,0
prawdobienstwo odejścia,0.200608
Liczebnosc populacji,1185.000000


In [41]:
prob_bellow_q90 = df[~df_h3]["Attrition_Flag"].value_counts()
d_above = {
"prawdobienstwo odejścia":prob_bellow_q90.iloc[1] / prob_bellow_q90.iloc[0],
 "Liczebnosc populacji":prob_bellow_q90.iloc[1] + prob_bellow_q90.iloc[0]
}
display(pd.DataFrame.from_dict(d_above,orient='index'))

,0
prawdobienstwo odejścia,0.190204
Liczebnosc populacji,8942.000000


In [44]:
display(pd.DataFrame.from_dict({
    "P-value dla t testu, porównanie miedzy powyżej 90 percentyla i poniżej":
st.ttest_ind(df[~df_h3]["Attrition_Flag_onehot"],df[df_h3]["Attrition_Flag_onehot"],
             alternative="larger")[1]
}, orient='index'))

,0
"P-value dla t testu, porównanie miedzy powyżej 90 percentyla i poniżej",0.739332


### P-vale jest wysokie, nie odrzucamy  H_0 dla t testu co oznacza że to jak długo klient był z bankiem nie wpływa znacząco na jego prawdopobieństwo odejścia. 

### Po przeprowadzeniu wstępnej analizy przechodzimy do modelowania

# Modelowanie 

### Z analizy wiemy które zmienne są interesujące i sugerują jakich modeli możemy użyć. Ważne jest aby wybrać model który jest odporny na nie zbalansowane dane. 

### Należy sobie teraz zdać pytanie czego oczekujemy od modelu żeby model był interpretowalny czy żeby zwracał jak nalepsze wyniki ponieważ mamy tutaj trade-off wyjaśnialny model nie będzie miał dużej prezycji czy celności natomias model z dużą precyzją i celnoścą może być nie do zintepretowania. Biorąc to pod uwagę oraz wcześniejsza analize stworzymy wybrane modele: 

1. Drzewo decyzyjne 
2. Regresje logistyczną 
3. Naive bayes
4. 

